In [1]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json

In [2]:
# purge all tables first
db.session.close()
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [3]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# post miguel
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [4]:
# post the rest of the users
users = [
    ('A', 'TPN@example.com'),
    ('B', 'AngryFalco@example.com'),
    ('C', 'Sunrisebanana@example.com'),
    ('D', 'Ptolemy@example.com'),
    ('E', 'Vik@example.com'),
    ('F', 'Kevin@example.com'),
    ('G', 'Spaceghost@example.com'),
    ('H', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })

    r = requests.post(url, data=payload, headers=headers)

In [5]:
# get miguel's token
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']

# miguel creates tournament\
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'A',
    'B',
    'C',
    'D',
    'E',
    'F',
    'G',
    'H',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

# create a json payload from the tuple list
url = 'http://127.0.0.1:5000/api/created-tournaments/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "users":usernames,
    "seeds":seeds,
    "tournament_name":"Fall Charity LAN 2018 Melee Singles"
})
r = requests.post(url, data=payload, headers=headers, auth=("miguel", 'python'))
print(r.content)

b'{"tournament_id":1}\n'


In [6]:
# print all the tables for good measure

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query

print('#'*68)
print()

print('Users\' table')
df = pd.read_sql_query(q, cnx)
print(df)
print()
for table in ['tournament', 'bracket', 'bracket_entrants', 'round', 'match']:
    print('#'*68)
    print()

    print(f'{table} table')
    print(pd.read_sql_query(f'select * from {table}', cnx))
    print()

####################################################################

Users' table
   id username                      email  \
0   1   miguel           miguel@gmail.com   
1   2        A            tpn@example.com   
2   3        B     angryfalco@example.com   
3   4        C  sunrisebanana@example.com   
4   5        D        ptolemy@example.com   
5   6        E            vik@example.com   
6   7        F          kevin@example.com   
7   8        G     spaceghost@example.com   
8   9        H        burnaby@example.com   

                                       password_hash about_me  \
0  pbkdf2:sha256:150000$fI889Rn3$515f691e94ff1958...     None   
1  pbkdf2:sha256:150000$wa8cpV2b$3023c94cda1b958d...     None   
2  pbkdf2:sha256:150000$Oz7udf1F$2a841f3c10757942...     None   
3  pbkdf2:sha256:150000$IXpK1TVP$26368b8bbc520bff...     None   
4  pbkdf2:sha256:150000$CfBTdPua$79abc7dbf738d448...     None   
5  pbkdf2:sha256:150000$Sxor0NI2$48648dcce6f7343a...     None   
6  pbkdf2:s

In [7]:
# get miguel's token
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzMzMzM0MiwiZXhwIjoxNTgzMzMzOTQyfQ.eyJpZCI6MX0.WUWwANycH7JJKbnsgv5OCg-BFNQ7pSI-o-ObmmY3KP8QID3MK-ee37rhDppS2aOwsgE1t3vv6xyDmVqNnfXFwA'

In [8]:
# get miguel's token
mid = 1
url = f'http://127.0.0.1:5000/api/match/{mid}/update'
r = requests.post(url, auth=(tkn, 'unused'))
r

<Response [200]>

In [9]:
url

'http://127.0.0.1:5000/api/match/1/update'

In [10]:
json.loads(r.content)

{'id': 1,
 'loser_to': 10,
 'match_winner': None,
 'round': 1,
 'u1': 9,
 'u2': 5,
 'user_1_score': None,
 'user_2_score': None,
 'uuid': '25f87bea-3321-4b6c-b2c3-930e3f7aed9f',
 'winner_to': 5}

In [11]:
url

'http://127.0.0.1:5000/api/match/1/update'